# Keras Code for training CNN on the SETI signal images
    1. Get the input data in the format of Numpy ndarrays in greyscale. Split the data into training and testing set. Probably 0.9:0.1
    2. We need to decide on the design parameters of the network
    3. Train the classifier
    4. Test the classifier

In [1]:
import os
import zipfile
import ibmseti
import keras
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [2]:
mydatafolder = os.path.join(os.environ['PWD'],'Machine_Contact_Enterprise_SETI')
zz = zipfile.ZipFile(mydatafolder + '/' + 'basic4.zip')
basic4list = zz.namelist()[1:]

In [3]:
def get_spectral_ndarray(filename):
    img = ibmseti.compamp.SimCompamp(zz.open(filename).read())
    return (img.get_spectrogram(),img.header()["signal_classification"])

In [4]:
def load_training_data():
    x_list = []
    y_list = []
    for filename in basic4list[0:10]:
        ans = get_spectral_ndarray(filename)
        x_list.append(ans[0])
        y_list.append(ans[1])
    return (np.array(x_list),np.array(y_list))

In [5]:
seed = 7
np.random.seed(seed)

In [6]:
# load training data
(X_train, y_train) = load_training_data()
#(X_test, y_test) = load_test_data()

In [7]:
# one-hot encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
print np.array(le.fit_transform(y_train))
y_train = np_utils.to_categorical(np.array(le.fit_transform(y_train)))

[1 3 2 3 0 2 0 3 3 0]


In [8]:
print y_train
#for i in range(len(y_train)):
#    if y_train[i] == 'narrowbanddrd'

[[ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 1.  0.  0.  0.]]


In [ ]:
# normalize the data
X_train = X_train.astype('float32')
X_train = X_train / 255.0

In [ ]:
# clean noise from the data
X.shape
average_noise=np.zeros(noise[0].shape)
for n in noise:
    average_noise += n/1000


In [ ]:
firstfile = basic4list[0]
aca = ibmseti.compamp.SimCompamp(zz.open(firstfile).read())
aca.header() # this shows the classification
spectrogram = aca.get_spectrogram()
spectrogram = spectrogram - noise - np.mean(spectrogram)
fig, ax = plt.subplots(figsize=(10, 5))
ax.imshow(np.log(spectrogram),  aspect = 0.5*float(spectrogram.shape[1]) / spectrogram.shape[0])

In [10]:
!ls Machine_Contact_Enterprise_SETI/

basic4_test.zip				  public_list_basic_v2_26may_2017.csv
basic4.zip				  watson_results.pickle
cleanPngs				  watson_scores.csv
data					  zipfiles
public_list_basic_testset_1june_2017.csv


In [29]:
# define CNN model
from keras.layers import Input
model = Sequential()
model.add(Input(shape=(1, 32, 6144)))
model.add(Conv2D(32, (3, 3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(50, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

TypeError: The added layer must be an instance of class Layer. Found: Tensor("input_2:0", shape=(?, 1, 32, 6144), dtype=float32)

In [13]:
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

(32, 6144)

In [ ]:
model.fit(X_train, y_train, validation_split=0.1 epochs=epochs, batch_size=150)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [19]:
!pip install --upgrade keras

    100% |████████████████████████████████| 204kB 3.8MB/s 
    100% |████████████████████████████████| 3.1MB 353kB/s 
    100% |████████████████████████████████| 256kB 3.2MB/s 
Requirement already up-to-date: six in /usr/local/src/bluemix_jupyter_bundle.v46/notebook/lib/python2.7/site-packages (from keras)
Requirement already up-to-date: numpy>=1.9.1 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s18f-42b3305d048694-5fca760d00b1/.local/lib/python2.7/site-packages (from theano->keras)
Requirement already up-to-date: scipy>=0.14 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s18f-42b3305d048694-5fca760d00b1/.local/lib/python2.7/site-packages (from theano->keras)
  Running setup.py bdist_wheel for keras ... - \ done
  Stored in directory: /gpfs/fs01/user/s18f-42b3305d048694-5fca760d00b1/.cache/pip/wheels/48/82/42/f06a8c03a8f95ada523a81ba723e89f059693e6ad868d09727
  Running setup.py bdist_wheel for theano ... - \ | / - \ | done
  Stored in directory: /gpfs/fs01/user/s